# Idea

Monitoring popular russian IT resurce and monitoring any feadbacks about company

# Imports

In [1]:
import requests
import json
import pandas as pd
import redis
from collections import defaultdict
from bs4 import BeautifulSoup

# DB connect 

In [2]:
import redis
connect = redis.Redis(host='redis', port=6379)

# Data process and save to BD

In [3]:
url = "https://vc.ru/claim"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
article_links = []
for tag in soup.find_all("a"):
    class_names = tag.get("class", [])
    if "content-link" in class_names:
        article_links.append(tag["href"])

storage = defaultdict(dict)
for link in article_links:
    storage[link]["link"] = link
    response = requests.get(link)
    article = BeautifulSoup(response.text, 'html.parser')
    for tag in article.find_all("h1"):
        storage[link]["name"] = tag.text.replace("\n", " ")
    text = ""
    for tag in article.find_all("div"):
        class_names = tag.get("class", []) 
        if "content--full" in class_names:
            text += tag.text
            print
    storage[link]["text"] = text.replace("\n", " ")
    connect.set(link, json.dumps(storage[link]))

In [5]:
df = pd.DataFrame(storage.values()).dropna()
df.sample(min(10, len(df)))

,link,name,text
7,https://vc.ru/claim/363769-i-snova-poteryana-p...,И снова потеряна посылка ...,Ровно через 2 недели после потерянной Boxber...
1,https://vc.ru/claim/363642-tinkoff-bank-psevdo...,«Тинькофф банк»: псевдо з...,На карте были отключены покупки в интернете....
2,https://vc.ru/claim/363636-kak-ya-napisal-teks...,Как я написал текст для к...,У пользователей изучения этих курсов английс...
3,https://vc.ru/claim/363819-kriptobirzha-exmo-s...,Криптобиржа Exmo списала ...,Если вы торгуете криптовалютой или хотите на...
8,https://vc.ru/claim/358097-kak-sitilink-isport...,Как ситилинк испортил мой...,В конце января покупал себе Macbook M1 Air в...
6,https://vc.ru/claim/363744-rostelekom,Ростелеком,Расскажу я вам веселую (нет) историю про Рос...
4,https://vc.ru/claim/363610-sliyanie-kompaniy-i...,Слияние компаний и потеря...,Есть у меня счет чуть больше года. Акции по ...
12,https://vc.ru/claim/363378-aliexpress-moshenni...,AliExpress – мошенничеств...,Многие наверняка видели на главной страничке...
0,https://vc.ru/claim/89692-otkazhites-ot-emociy...,Откажитесь от эмоций и оп...,"Чем конструктивнее — тем больше шансов, что ..."
9,https://vc.ru/claim/363724-zdravstvuyte-peresh...,"Здравствуйте,перешла с Ме...","Была произведена оплата по лицевому счёту, в..."


# Get from DB and find companies

In [6]:
companies = ["Яндекс", "Yandex", "Apple", "Ozon", "Озон", "Тинькофф", "Альфа"]
for link in connect.keys():
    article = json.loads(connect[link])
    mentiond = []
    for company in companies:
        if company.lower() in article.get("name", "").lower():
            print(f"Founded {company} company!")
            mentiond.append(company)
    
    article["company"] = mentiond
    connect[link] = json.dumps(article)

Founded Озон company!
Founded Тинькофф company!
Founded Альфа company!


# Check result

In [7]:
data = [json.loads(connect[link]) for link in connect.keys()]
df = pd.DataFrame(data).dropna()
df.sample(min(10, len(df)))

,link,name,text,company
3,https://vc.ru/claim/363724-zdravstvuyte-peresh...,"Здравствуйте,перешла с Ме...","Была произведена оплата по лицевому счёту, в...",[]
1,https://vc.ru/claim/363531-internet-magazin-ne...,\n Интернет магазин не возв...,\n\nРассказываю историю.\n\n\n\n\n\n\n\n18\n18...,[]
2,https://vc.ru/claim/363744-rostelekom,Ростелеком,Расскажу я вам веселую (нет) историю про Рос...,[]
12,https://vc.ru/claim/363819-kriptobirzha-exmo-s...,Криптобиржа Exmo списала ...,Если вы торгуете криптовалютой или хотите на...,[]
5,https://vc.ru/claim/89692-otkazhites-ot-emociy...,Откажитесь от эмоций и оп...,"Чем конструктивнее — тем больше шансов, что ...",[]
10,https://vc.ru/claim/358097-kak-sitilink-isport...,Как ситилинк испортил мой...,В конце января покупал себе Macbook M1 Air в...,[]
6,https://vc.ru/claim/363848-ozon-otkazyvaet-v-p...,Озон отказывает в предост...,"Как многим тут известно, с недавнего времени...",[Озон]
11,https://vc.ru/claim/363378-aliexpress-moshenni...,AliExpress – мошенничеств...,Многие наверняка видели на главной страничке...,[]
8,https://vc.ru/claim/363769-i-snova-poteryana-p...,И снова потеряна посылка ...,Ровно через 2 недели после потерянной Boxber...,[]
13,https://vc.ru/claim/363418-alfa-bank-postoyann...,Альфа банк постоянно запр...,Добрый день! 82 82 комментария ...,[Альфа]
